In [2]:
from Data_Loader import load_data
from Train import trainCox_nnet

import torch
import numpy as np
import pandas as pd

In [3]:
dtype = torch.FloatTensor
''' Net Settings'''
In_Nodes = 55553 ###number of genes
Hidden_Nodes = 100 ###number of hidden nodes
Out_Nodes = 30 ###number of hidden nodes in the last hidden layer
''' Initialize '''
Initial_Learning_Rate = [0.03, 0.01, 0.001, 0.00075]
L2_Lambda = [0.1, 0.01, 0.005, 0.001]
num_epochs = 300 ###for grid search
Num_EPOCHS = 2000 ###for training
###sub-network setup
Dropout_Rate = [0.7]

In [4]:
from Data_extraction import output_df2, x_df2  

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\anais\\Documents\\2-Imperial\\0-Research-Project\\4-NN\\Cox-nnet model\\output\\clinical.tsv'

In [ ]:
data = pd.concat([x_df2,output_df2],axis=1)
x, ytime, yevent, age = load_data(data, dtype)

In [ ]:
x_train, ytime_train, yevent_train, age_train = x[0:107], ytime[0:107], yevent[0:107], age[0:107]
x_valid, ytime_valid, yevent_valid, age_valid = x[107:142], ytime[107:142], yevent[107:142], age[107:142]
x_test, ytime_test, yevent_test, age_test = x[142:len(ytime)], ytime[142:len(ytime)], yevent[142:len(ytime)], age[142:len(ytime)]

In [ ]:
opt_l2_loss = 0
opt_lr_loss = 0
opt_loss = torch.Tensor([float("Inf")])
opt_c_index_va = 0
opt_c_index_tr = 0

In [ ]:
for l2 in L2_Lambda:
	for lr in Initial_Learning_Rate:
		loss_train, loss_valid, c_index_tr, c_index_va = trainCox_nnet(x_train, age_train, ytime_train, yevent_train, \
																x_valid, age_valid, ytime_valid, yevent_valid, \
																In_Nodes, Hidden_Nodes, Out_Nodes, \
																lr, l2, num_epochs, Dropout_Rate)
		if loss_valid < opt_loss:
			opt_l2_loss = l2
			opt_lr_loss = lr
			opt_loss = loss_valid
			opt_c_index_tr = c_index_tr
			opt_c_index_va = c_index_va
		print ("L2: ", l2, "LR: ", lr, "Loss in Validation: ", loss_valid)

Loss in Train:  tensor([nan], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([nan], grad_fn=<ViewBackward0>)
L2:  0.1 LR:  0.03 Loss in Validation:  tensor([nan], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([-inf], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([nan], grad_fn=<ViewBackward0>)
L2:  0.1 LR:  0.01 Loss in Validation:  tensor([nan], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([7.3616], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6717], grad_fn=<ViewBackward0>)
L2:  0.1 LR:  0.001 Loss in Validation:  tensor([2.6501], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.7362], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6717], grad_fn=<ViewBackward0>)
L2:  0.1 LR:  0.00075 Loss in Validation:  tensor([2.6500], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([nan], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([nan], grad_fn=<ViewBackward0>)
L2:  0.01 LR:  0.03 Loss in Validation:  tensor([nan], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([-inf], 

In [ ]:
###train Cox-PASNet with optimal hyperparameters using train data, and then evaluate the trained model with test data
###Note that test data are only used to evaluate the trained Cox-nnet
loss_train, loss_test, c_index_tr, c_index_te = trainCox_nnet(x_train, age_train, ytime_train, yevent_train, \
							x_test, age_test, ytime_test, yevent_test,  \
							In_Nodes, Hidden_Nodes, Out_Nodes, \
							opt_lr_loss, opt_l2_loss, Num_EPOCHS, Dropout_Rate)
print ("Optimal L2: ", opt_l2_loss, "Optimal LR: ", opt_lr_loss)
print("C-index in Test: ", c_index_te)


Loss in Train:  tensor([5.3471], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6703], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6623], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6717], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6717], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6630], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6620], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6652], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6741], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6657], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([3.6655], grad_fn=<ViewBackward0>)
Optimal L2:  0.01 Optimal LR:  0.00075
C-index in Test:  tensor(0.4857)


In [ ]:
print(np.where(yevent_train==0))
# print(yevent_valid)
# print(yevent_test)

(array([], dtype=int64), array([], dtype=int64))


In [ ]:
print(opt_l2_loss, opt_lr_loss)

0.01 0.00075
